# Web Data Scraping

[Spring 2023 ITSS Mini-Course](https://www.colorado.edu/cartss/programs/interdisciplinary-training-social-sciences-itss/mini-course-web-data-scraping) — ARSC 5040  
[Brian C. Keegan, Ph.D.](http://brianckeegan.com/)  
[Assistant Professor, Department of Information Science](https://www.colorado.edu/cmci/people/information-science/brian-c-keegan)  
University of Colorado Boulder  

Copyright and distributed under an [MIT License](https://opensource.org/licenses/MIT)

## Class outline

* **Week 1**: Introduction to Jupyter, browser console, structured data, ethical considerations
* **Week 2**: Scraping HTML with `requests` and `BeautifulSoup`
* **Week 3**: Scraping web data with Selenium
* **Week 4**: Scraping the Internet Archive and Wikipedia APIs
* **Week 5**: Scraping the Reddit and Mastodon APIs

## Acknowledgements

This course will draw on resources built by myself and [Allison Morgan](https://allisonmorgan.github.io/) for the [2018 Summer Institute for Computational Social Science](https://github.com/allisonmorgan/sicss_boulder), which were in turn derived from [other resources](https://github.com/simonmunzert/web-scraping-with-r-extended-edition) developed by [Simon Munzert](http://simonmunzert.github.io/) and [Chris Bail](http://www.chrisbail.net/). 

Thank you also to Professor [Terra KcKinnish](https://www.colorado.edu/economics/people/faculty/terra-mckinnish) for coordinating the ITSS seminars.

This notebook is adapted from excellent notebooks in Dr. [Cody Buntain](http://cody.bunta.in/)'s seminar on [Social Media and Crisis Informatics](http://cody.bunta.in/teaching/2018_winter_umd_inst728e/) as well as the [PRAW documentation](https://praw.readthedocs.io/en/latest/).

## Class 4 goals

* Sharing accomplishments and challenges with last week's material
* Accessing an API with `requests`
* Retrieiving historical web data from the Internet Archive
* Why you don't want to write a parser for Wikipedia's data
* Fundamentals of retrieving information from Wikipedia's API
* EDA with data from Wikipedia's API

We'll need a few common libraries for all these examples.

In [ ]:
# Lets us talk to other servers on the web
import requests

# APIs spit out data in JSON
import json

# Use BeautifulSoup to parse some HTML
from bs4 import BeautifulSoup

# Safetly quoting strings for URLs
from urllib.parse import unquote, quote

# Handling dates and times
from datetime import datetime

# DataFrames!
import pandas as pd
import numpy as np

# Data visualization
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sb

## Scraping the Internet Archive's Wayback Machine

Now we'll leave some of the ethically-fraught methods of web scraping behind. The Internet Archive maintains the "[Wayback Machine](https://www.archive.org/web/)" where old versions of websites are stored. Some of my favorites:

* [CNN in June 2000](https://web.archive.org/web/20000815052826/http://www.cnn.com/)
* [Facebook in August 2004](https://web.archive.org/web/20040817020419/http://www.facebook.com/)
* [Apple in April 1997](https://web.archive.org/web/19970404064444/http://www.apple.com:80/)

In these URLs above, there is a numeric identifier corresponding to the timestamp when the image of the website was captured. How do we know when the Wayback Machine archived a webpage? There's a free and open API!

### Using the Wayback Machine API

The simplest API request we can make asks for the most recent snapshot of a webpage archived by the Wayback Machine.

In [ ]:
wb_url = 'http://archive.org/wayback/available?url=facebook.com'

wb_response = requests.get(wb_url)

wb_response.json()

This response tells us the timestamp and location of this snapshot, which we could then go retrieve and parse.

In [ ]:
wb_response_json = wb_response.json()

recent_fb_wb_url = wb_response_json['archived_snapshots']['closest']['url']

recent_fb_wb_response = requests.get(recent_fb_wb_url)

Get the raw text out, soupify, and look for links. For some reason all the links in this snapshot are in German.

In [ ]:
recent_fb_wb_raw = recent_fb_wb_response.text

recent_fb_wb_soup = BeautifulSoup(recent_fb_wb_raw)

[link.text for link in recent_fb_wb_soup.find_all('a')]

We can also ask for the most recent snapshot of a webpage around a specific date. Let's ask the Wayback Machine for a snapshot of Facebook around February 1, 2008.

In [ ]:
wb_url = 'http://archive.org/wayback/available?url=facebook.com&timestamp=20080201'

wb_response = requests.get(wb_url)

wb_response_json = wb_response.json()

wb_response_json

Note that this is a relatively deep JSON object we have to navigate into to access information like the Wayback URL or the timestamp of the snapshot. The closest snapshot to February 1, 2008 was January 30, 2008. We use the [`datetime.strptime`](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior) function to turn this numeric string that we recognize as a timestamp into a datetime object.

In [ ]:
wb_response_json['archived_snapshots']['closest']['timestamp']

In [ ]:
print(datetime.strptime(wb_response_json['archived_snapshots']['closest']['timestamp'],
                        '%Y%m%d%H%M%S'))

As before, we could scrape out the links on this 2008 version of the page.

In [ ]:
# Find the old URL
fb_wb_url = wb_response_json['archived_snapshots']['closest']['url']

# Go get the archived snapshot from the Wayback Machine
fb_wb_response = requests.get(fb_wb_url)

# Get the text from the response
fb_wb_raw = fb_wb_response.text

# Soup-ify
fb_wb_soup = BeautifulSoup(fb_wb_raw)

# Make a list of the text of the links
[link.text for link in fb_wb_soup.find_all('a')]

### Scraping historical web pages

A current project I am working on is exploring how social media platforms' terms of service have evolved over time. Let's start with Facebook's terms of service and privacy policy.

In [ ]:
fb_tos = 'http://www.facebook.com/terms.php'
fb_pp = 'http://www.facebook.com/policy.php'

We will take advantage of the [`date_range`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.date_range.html) fuction in `pandas` to generate a range of dates between January 2005 and January 2019.

In [ ]:
dates_list = pd.date_range(start='2005-01-01',end='2021-03-01',freq='M')
dates_list

We'll use [`datetime.strftime`](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior) (the inverse of `strptime`) to make these date objects into specifically-formatted strings that we can format into a URL.

In [ ]:
# Take the first datetime object and turn it into a string
datetime.strftime(dates_list[0],'%Y%m%d')

Use string formatting to put the `fb_tos` URL and formatted timestamp into a request to the Wayback Machine.

In [ ]:
date_str = datetime.strftime(dates_list[0],'%Y%m%d')

wb_api_url = 'https://archive.org/wayback/available?url={0}&timestamp={1}'
wb_api_url_formatted = wb_api_url.format(fb_tos,date_str)

print(wb_api_url_formatted)

Make the request to the Wayback Machine to get the URL and timestamp of the Wayback Machine's closest snapshot of Facebook's Terms of Service before January 31, 2005.

In [ ]:
wb_api_response = requests.get(wb_api_url_formatted)

wb_api_response.json()

Parse the markup of this old version.

In [ ]:
# Find the old URL
wb_fb_old_url = wb_api_response.json()['archived_snapshots']['closest']['url']

# Go get the archived snapshot from the Wayback Machine
wb_fb_raw = requests.get(wb_fb_old_url).text

# Soup-ify
wb_fb_soup = BeautifulSoup(wb_fb_raw)

# Find the content element and get the text out
wb_fb_terms_str = wb_fb_soup.find('div',{'id':'content'}).text.strip()

# Inspect
wb_fb_terms_str

We could use a really dumb stemmer, [`.split()`](https://docs.python.org/3.7/library/stdtypes.html#str.split) to count the number of words in these terms.

In [ ]:
len(wb_fb_terms_str.split())

Write a loop to find a snapshot of Facebook's ToS each month in our `dates_list`. 

In [ ]:
def get_urls(url_str,start_date='2005-01-01',end_date='2021-03-01',freq='M'):
    
    # Make the list of dates
    date_l = pd.date_range(start_date,end_date,freq=freq)
    
    # Create an empty container to store our data
    urls = dict()

    # For each date in the list of dates
    for date in date_l:
        
        # Turn the date object back into a string
        date_str = datetime.strftime(date,'%Y%m%d%H%M%S')
        
        # Define the API URL request to the Wayback machine
        wb_api_url = 'http://archive.org/wayback/available?url={0}&timestamp={1}'
        
        # Format the API URL with the URL of the website and the closest datetime
        wb_api_request = wb_api_url.format(url_str,date_str)
        
        # Make the request
        r = requests.get(wb_api_request).json()

        # Check if the returned request has all the right parts (this is probably overkill)
        if 'archived_snapshots' in r.keys():
            if 'closest' in r['archived_snapshots'].keys():
                if 'url' in r['archived_snapshots']['closest'].keys():
                    
                    # If it does have all the right parts, get the URL
                    _url = r['archived_snapshots']['closest']['url']
                    
                    # Get the timestamp
                    _timestamp = r['archived_snapshots']['closest']['timestamp']
                    
                    # Save to our URL dictionary with the timestamp of the snapshot as key, the url as value
                    urls[_timestamp] = _url
    return urls

Run our function to make a dictionary of keys returning the Wayback Machine URLs for each month's version of the terms of service. We'll write a loop to get the Terms for each snapshot and count the words. 

This will take a few minutes. 

I've coverted the code block into a "Raw" cell to prevent accidental execution. You can always turn it into a "Code" cell if you really want to run it.

In [ ]:
# Get the list of timestamps and URLs for each monthly version of the Terms of Service
fb_terms_d = get_urls('https://www.facebook.com/terms.php')

# Create an empty container to store our data
fb_terms_wordcount = {}

# Loop through the fb_terms_d dictionary
for timestamp,url in fb_terms_d.items():

    # Get the raw HTML from the Wayback Machine
    raw = requests.get(url).text
    
    # Soup-ify
    soup = BeautifulSoup(raw)
    
    # Find the content of the TOS
    content = soup.find('div',{'id':'content'}).text.strip()
    
    # Split the content into words, count the number of words, save to the container
    fb_terms_wordcount[timestamp] = len(content.split())
    
# Write to disk
with open('facebook_tos_archive.json','w') as f:
    json.dump(fb_terms_wordcount,f)

In [ ]:
with open('facebook_tos_archive.json','w') as f:
    json.dump(fb_terms_wordcount,f)

To avoid having everyone hit the Internet Archive server with the same requests, you can also load this file with the same data.

In [ ]:
with open('facebook_tos_archive.json','r') as f:
    fb_terms_wordcount2 = json.load(f)

Visualize the changes in the size of Facebook's Terms of Service over time.

In [ ]:
# Turn the dictionary into a pandas Series
fb_terms_s = pd.Series(fb_terms_wordcount2)

# Conver the index to datetime objects
fb_terms_s.index = pd.to_datetime(fb_terms_s.index)

# Plot
ax = fb_terms_s.plot()

# Make the x-tick labels less weird
# ax.set_xticklabels(range(2004,2021,2),rotation=0,horizontalalignment='center')

# Always label your axes
ax.set_ylabel('Word count');

## Scraping Wikipedia

Consider the Wikipedia page for [Elizabeth II](https://en.wikipedia.org/wiki/Elizabeth_II). This seems like a relatively straightforward webpage to scrape out the hyperlinks to other articles or to compare the content to other presidential biographies. However, Wikipedia also preserves the [history of every revision made to this article](https://en.wikipedia.org/w/index.php?title=Elizabeth_II&action=history) going back to the first (available) revisions in 2001, like [this](https://en.wikipedia.org/w/index.php?title=Elizabeth_II&oldid=249061). Thinking back to the Oscars example, it seems promising to find the "oldid" values and visit each revision's webpage to parse the content out. However, Wikipedia will give you much of this revision history data for free through its [application programming interface](http://en.wikipedia.org/w/api.php) (API).

### Current content
We can use `requests` to get the current HTML markup of an article from the API, for example.

In [ ]:
# Where the API server lives
query_url = "https://en.wikipedia.org/w/api.php"

# An empty dictionary to store our query parameters
query_params = {}

# We want to parse the content of a page
query_params['action'] = 'parse'

# Which page?
query_params['page'] = 'Elizabeth II'

# We want the text
query_params['prop'] = 'text'

# Ignore the edit buttons and table of contents
query_params['disableeditsection'] = 1
query_params['disabletoc'] = 1

# Get the results back as JSON
query_params['format'] = 'json'

# Format the data in an easier-to-parse option
query_params['formatversion'] = 2

We have only set up our request to the API, but not sent it or received the data back.

In [ ]:
json_response = requests.get(url = query_url, params = query_params).json()

What's waiting inside? A dictionary of dictionaries. The inner dictionary has keys for the title of the page we requested ("George H. W. Bush"), the pageid (a numeric identifier), and the text of the article.

In [ ]:
json_response['parse'].keys()

We could count the number of links in the article.

In [ ]:
ghwb_soup = BeautifulSoup(json_response['parse']['text'])

ghwb_soup.find_all('a')[:5]

Or the content of the article.

In [ ]:
ghwb_soup.find_all('p')[:5]

### Revision history

There is also an API endpoint for the revision history of this article that contains metadata about the who and when of previous changes.

In [ ]:
# Where the API server lives
query_url = "https://en.wikipedia.org/w/api.php"

# An empty dictionary to store our query parameters
query_params = {}

# We want to query properties of a page
query_params['action'] = 'query'

# Which page?
query_params['titles'] = 'Elizabeth II'

# We want the revisions
query_params['prop'] = 'revisions'

# In particular, we want the revision ids, users, comments, timestamps
query_params['rvprop'] = 'ids|userid|comment|timestamp|user|size|sha1'

# Get 500 revisions
query_params['rvlimit'] = 500

# Start old and go newer
query_params['rvdir'] = 'newer'
    
# Get the results back as JSON
query_params['format'] = 'json'

# Format the data in an easier-to-parse option
query_params['formatversion'] = 2

Make the request.

In [ ]:
json_response = requests.get(url = query_url, params = query_params).json()

Inspect this `json_response`. This returns a dictionary with both "continue" and "query" keys. The continue indicates there are more than 500 revisions present in the article's history and provides an index for the next query to pick up from. The query contains the revision history we care about—buried a bit in a nested data structure of lists and dictionaries, but we eventually get to the "revisions" list of dictionaries with the revision histories.

In [ ]:
revisions = json_response['query']['pages'][0]['revisions']
revisions[:3]

Convert to a DataFrame.

In [ ]:
rev_df = pd.DataFrame(revisions)
rev_df.head()

Plot out how the size of the article changed over the first 500 revisions.

In [ ]:
ax = rev_df.plot(y='size',legend=False)
ax.set_ylabel('Size (bytes)')
ax.set_xlabel('Revision')
ax.set_xlim((0,500))

Or count how many times an editor made a contribution.

In [ ]:
rev_df['user'].value_counts().head()

### Pageview activity

There is are [API endpoints](https://wikitech.wikimedia.org/wiki/Analytics/AQS/Pageviews) for retrieving data on the number of times a Wikipedia article was accessed. Let's use the pageviews API ([docs](https://wikimedia.org/api/rest_v1/#/Pageviews data/)). Unfortunately, these article-level data are not stratifed by geography (by state, country, *etc*.) and are only available in the aggregate. This API only has data available back to July 2015, although there are some alternative [pageview definitions](https://meta.wikimedia.org/wiki/Research:Page_view) with data [available back to 2007](https://dumps.wikimedia.org/other/pagecounts-raw/). 

In [ ]:
# This is the string for accessing the API with some variables we'll customize
pv_api_str = "https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/{lang}.wikipedia/all-access/user/{article}/daily/{start}/{end}"

# Customizing the pv_api_str to get information about Elizabeth II from 1 July 2015 to 1 January 2023
retrieve_str = pv_api_str.format(
    lang = 'en',
    article = quote('Elizabeth II'),
    start = '20150701',
    end = '20230101'
)

# Make the request
pv_response = requests.get(retrieve_str,headers={'user-agent':'brian.keegan@colorado.edu'}).json()

# Inspect
pv_response['items'][:5]

Convert into a pandas `DataFrame` and cleanup.

In [ ]:
# Cast to a DataFrame
e2_pv_df = pd.DataFrame(pv_response['items'])

# Parse the "timestamp" column into a datetime
e2_pv_df['timestamp'] = pd.to_datetime(e2_pv_df['timestamp'],format='%Y%m%d%H')

# Set timestamp as index and only return reviews
e2_pv_s = e2_pv_df.set_index('timestamp')['views']

# Inspect
e2_pv_s.head()

Visualize on log-scaled y-axis.

In [ ]:
ax = e2_pv_s.plot()

ax.set_yscale('log')
ax.set_ylabel('Pageviews')
ax.set_ylim((1e4,1e7))

Write a function and use it in a loop to get the pageviews for each of these redirects.

In [ ]:
def get_pageviews(article_title,lang='en',start='20150701',end='20230101',headers={'user-agent':'brian.keegan@colorado.edu'}):
    # This is the string for accessing the API with some variables we'll customize
    _str = "https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/{lang}.wikipedia/all-access/user/{article}/daily/{start}/{end}"

    # Customizing the pv_api_str to get information about Elizabeth II from 1 July 2015 to 1 January 2023
    _retrieve_str = _str.format(
        lang = lang,
        article = quote(article_title),
        start = start,
        end = end
    )
    
    # Make the request
    _response = requests.get(_retrieve_str,headers=headers).json()
    
    # Cast to a DataFrame
    _pv_df = pd.DataFrame(pv_response['items'])

    # Parse the "timestamp" column into a datetime
    _pv_df['timestamp'] = pd.to_datetime(_pv_df['timestamp'],format='%Y%m%d%H')

    # Set timestamp as index and only return reviews
    _pv_s = _pv_df.set_index('timestamp')['views']
    
    return _pv_s
    

Test the function.

In [ ]:
get_pageviews('University of Colorado Boulder').plot()

There are many other parts of the very powerful Wikipedia API and scraping these APIs exposes much more metadata than parsing the HTML of these webpages, while also being easier on the servers hosting it. 

Check out the "Class 04 - Wikipedia functions and analysis" notebook as well as my [`wikifunctions`](https://github.com/brianckeegan/wikifunctions/) library.